# Librerías

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import os
import seaborn as sns

%matplotlib inline

# Datos

In [2]:
train = pd.read_csv('../data/prepro_train.csv')
test = pd.read_csv('../data/prepro_test.csv')

In [3]:
train.head()

,date_number,product_id,block_id,stock,std_stock,family_id,subfamily_id,size_id,color_id,position_max,position_min,std_position,category_id,price,sales
0,0,310130,1726,1383,34.811328,679611953,533441312,7,1,80.0,3.0,37.194534,3,12.95,33
1,0,1178388,592,60,2.160247,732697347,691762817,4,1,19.0,19.0,0.000000,1,49.95,0
2,0,1561460,1625,2373,55.438769,396066037,520569701,5,1,38.0,3.0,14.808299,3,29.95,21
3,0,1874414,1135,1686,20.463906,744793598,811402796,6,1,190.0,12.0,59.873199,6,25.95,24
4,0,2436420,779,245,23.377339,768025921,665805124,5,1,NaN,NaN,NaN,0,25.95,0


In [4]:
test.head()

,date_number,product_id,block_id,stock,std_stock,family_id,subfamily_id,size_id,color_id,position_max,position_min,std_position,category_id,price,sales
0,71,151926,1969,636,33.103206,396066037,335531561,5,1,22.0,5.0,8.959787,2,25.95,-1
1,71,213413,1648,1190,36.225788,552529755,11509337,7,1,139.0,57.0,42.547711,2,19.95,-1
2,71,310130,1726,442,52.809180,679611953,533441312,7,1,46.0,46.0,0.000000,1,12.95,-1
3,71,455200,1400,86,6.831301,998145072,490222156,3,1,53.0,36.0,9.311283,2,29.95,-1
4,71,571044,1098,416,42.178063,831347344,750943270,4,2,205.0,154.0,26.462601,2,15.95,-1


In [5]:
n = 0
d_n = list()
day = list()
for date_n in range(0, 111):
    d_n.append(date_n)
    day.append(n)
    n += 1
    if n >= 7:
        n = 0
        
df_day = pd.DataFrame({'date_number': d_n, 'day': day})
df_day.head(10)

,date_number,day
0,0,0
1,1,1
2,2,2
3,3,3
4,4,4
5,5,5
6,6,6
7,7,0
8,8,1
9,9,2


In [6]:
train = train.join(df_day.set_index('date_number'), on='date_number', how='left')
test = test.join(df_day.set_index('date_number'), on='date_number', how='left')

In [7]:
test.head()

,date_number,product_id,block_id,stock,std_stock,family_id,subfamily_id,size_id,color_id,position_max,position_min,std_position,category_id,price,sales,day
0,71,151926,1969,636,33.103206,396066037,335531561,5,1,22.0,5.0,8.959787,2,25.95,-1,1
1,71,213413,1648,1190,36.225788,552529755,11509337,7,1,139.0,57.0,42.547711,2,19.95,-1,1
2,71,310130,1726,442,52.809180,679611953,533441312,7,1,46.0,46.0,0.000000,1,12.95,-1,1
3,71,455200,1400,86,6.831301,998145072,490222156,3,1,53.0,36.0,9.311283,2,29.95,-1,1
4,71,571044,1098,416,42.178063,831347344,750943270,4,2,205.0,154.0,26.462601,2,15.95,-1,1


#### Frequency encoding

Para las categorias `family_id` y `subfamily_id`

In [8]:
encoding_train = train.groupby('family_id').size()
encoding_train = encoding_train/len(train)
train['family_id'] = train.family_id.map(encoding_train)
encoding_train = train.groupby('subfamily_id').size()
encoding_train = encoding_train/len(train)
train['subfamily_id'] = train.subfamily_id.map(encoding_train)

encoding_test = test.groupby('family_id').size()
encoding_test = encoding_test/len(test)
test['family_id'] = test.family_id.map(encoding_test)
encoding_test = test.groupby('subfamily_id').size()
encoding_test = encoding_test/len(test)
test['subfamily_id'] = test.subfamily_id.map(encoding_test)

In [9]:
train['stock_lag1'] = train.groupby(['product_id'])['stock'].shift(1)
test['stock_lag1'] = test.groupby(['product_id'])['stock'].shift(1)
train['stock_lead1'] = train.groupby(['product_id'])['stock'].shift(-1)
test['stock_lead1'] = test.groupby(['product_id'])['stock'].shift(-1)

train['std_stock_shift1'] = train[['stock', 'stock_lag1', 'stock_lead1']].std(axis=1)
test['std_stock_shift1'] = test[['stock', 'stock_lag1', 'stock_lead1']].std(axis=1)

train['stock_lag2'] = train.groupby(['product_id'])['stock'].shift(2)
test['stock_lag2'] = test.groupby(['product_id'])['stock'].shift(2)
train['stock_lead2'] = train.groupby(['product_id'])['stock'].shift(-2)
test['stock_lead2'] = test.groupby(['product_id'])['stock'].shift(-2)

train['std_stock_shift2'] = train[['stock', 'stock_lag1', 'stock_lead1',
                                   'stock_lag2', 'stock_lead2']].std(axis=1)
test['std_stock_shift2'] = test[['stock', 'stock_lag1', 'stock_lead1',
                                   'stock_lag2', 'stock_lead2']].std(axis=1)

In [10]:
train['size_p_color'] = train.size_id * train.color_id
test['size_p_color'] = test.size_id * test.color_id

In [11]:
train['ratio_std_pos'] = train.std_position / max(train.std_position.max(), test.std_position.max())
test['ratio_std_pos'] = test.std_position / max(train.std_position.max(), test.std_position.max())

In [12]:
train['ratio_std_stock'] = train.std_stock / max(train.std_stock.max(), test.std_stock.max())
test['ratio_std_stock'] = test.std_stock / max(train.std_stock.max(), test.std_stock.max())

In [13]:
train['diff_position'] = train.position_max - train.position_min
test['diff_position'] = test.position_max - test.position_min

In [14]:
train.head()

,date_number,product_id,block_id,stock,std_stock,family_id,subfamily_id,size_id,color_id,position_max,...,stock_lag1,stock_lead1,std_stock_shift1,stock_lag2,stock_lead2,std_stock_shift2,size_p_color,ratio_std_pos,ratio_std_stock,diff_position
0,0,310130,1726,1383,34.811328,0.007130,0.000598,7,1,80.0,...,NaN,1311.0,50.911688,NaN,1305.0,43.405069,7,0.094515,0.053357,77.0
1,0,1178388,592,60,2.160247,0.049198,0.012761,4,1,19.0,...,NaN,60.0,0.000000,NaN,60.0,0.000000,4,0.000000,0.003311,0.0
2,0,1561460,1625,2373,55.438769,0.123964,0.023599,5,1,38.0,...,NaN,2376.0,2.121320,NaN,2355.0,11.357817,5,0.037629,0.084974,35.0
3,0,1874414,1135,1686,20.463906,0.110784,0.016021,6,1,190.0,...,NaN,1692.0,4.242641,NaN,1686.0,3.464102,6,0.152144,0.031366,178.0
4,0,2436420,779,245,23.377339,0.025903,0.004497,5,1,NaN,...,NaN,248.0,2.121320,NaN,248.0,1.732051,5,NaN,0.035832,NaN


In [15]:
max_std_train1 = train.groupby('product_id').max()['std_stock_shift1']
max_std_train1.name = 'max_std_stock_shift1'
max_std_test1 = test.groupby('product_id').max()['std_stock_shift1']
max_std_test1.name = 'max_std_stock_shift1'

train = train.join(max_std_train1, on='product_id', how='left')
test = test.join(max_std_test1, on='product_id', how='left')

train['ratio_std_shift1'] = train.std_stock_shift1 / train.max_std_stock_shift1
test['ratio_std_shift1'] = test.std_stock_shift1 / test.max_std_stock_shift1

train = train.drop('max_std_stock_shift1', axis=1)
test = test.drop('max_std_stock_shift1', axis=1)

max_std_train2 = train.groupby('product_id').max()['std_stock_shift2']
max_std_train2.name = 'max_std_stock_shift2'
max_std_test2 = test.groupby('product_id').max()['std_stock_shift2']
max_std_test2.name = 'max_std_stock_shift2'

train = train.join(max_std_train2, on='product_id', how='left')
test = test.join(max_std_test2, on='product_id', how='left')

train['ratio_std_shift2'] = train.std_stock_shift2 / train.max_std_stock_shift2
test['ratio_std_shift2'] = test.std_stock_shift2 / test.max_std_stock_shift2

train = train.drop('max_std_stock_shift2', axis=1)
test = test.drop('max_std_stock_shift2', axis=1)

In [16]:
train.head()

,date_number,product_id,block_id,stock,std_stock,family_id,subfamily_id,size_id,color_id,position_max,...,std_stock_shift1,stock_lag2,stock_lead2,std_stock_shift2,size_p_color,ratio_std_pos,ratio_std_stock,diff_position,ratio_std_shift1,ratio_std_shift2
0,0,310130,1726,1383,34.811328,0.007130,0.000598,7,1,80.0,...,50.911688,NaN,1305.0,43.405069,7,0.094515,0.053357,77.0,0.062963,0.058491
1,0,1178388,592,60,2.160247,0.049198,0.012761,4,1,19.0,...,0.000000,NaN,60.0,0.000000,4,0.000000,0.003311,0.0,0.000000,0.000000
2,0,1561460,1625,2373,55.438769,0.123964,0.023599,5,1,38.0,...,2.121320,NaN,2355.0,11.357817,5,0.037629,0.084974,35.0,0.054180,0.244738
3,0,1874414,1135,1686,20.463906,0.110784,0.016021,6,1,190.0,...,4.242641,NaN,1686.0,3.464102,6,0.152144,0.031366,178.0,0.023625,0.024590
4,0,2436420,779,245,23.377339,0.025903,0.004497,5,1,NaN,...,2.121320,NaN,248.0,1.732051,5,NaN,0.035832,NaN,1.000000,1.000000


In [17]:
train[train.product_id==151926]

,date_number,product_id,block_id,stock,std_stock,family_id,subfamily_id,size_id,color_id,position_max,...,std_stock_shift1,stock_lag2,stock_lead2,std_stock_shift2,size_p_color,ratio_std_pos,ratio_std_stock,diff_position,ratio_std_shift1,ratio_std_shift2
377662,59,151926,1969,620,17.163916,0.123964,0.03077,4,1,62.0,...,494.974747,NaN,1232.0,381.289042,4,0.068898,0.026308,53.0,1.000000,1.000000
385799,60,151926,1969,1320,29.742757,0.123964,0.03077,5,1,64.0,...,381.289042,NaN,1156.0,315.205753,5,0.070392,0.045588,54.0,0.770320,0.826685
393946,61,151926,1969,1232,28.296178,0.123964,0.03077,5,1,65.0,...,82.073138,620.0,1044.0,273.504662,5,0.071050,0.043371,55.0,0.165813,0.717316
402012,62,151926,1969,1156,26.765256,0.123964,0.03077,5,1,65.0,...,94.572723,1320.0,NaN,117.075474,5,0.071696,0.041024,55.0,0.191066,0.307052
410007,63,151926,1969,1044,24.839697,0.123964,0.03077,5,1,63.0,...,79.195959,1232.0,NaN,94.572723,5,0.066584,0.038073,53.0,0.160000,0.248034


In [19]:
train.columns.tolist()

['date_number',
 'product_id',
 'block_id',
 'stock',
 'std_stock',
 'family_id',
 'subfamily_id',
 'size_id',
 'color_id',
 'position_max',
 'position_min',
 'std_position',
 'category_id',
 'price',
 'sales',
 'day',
 'stock_lag1',
 'stock_lead1',
 'std_stock_shift1',
 'stock_lag2',
 'stock_lead2',
 'std_stock_shift2',
 'size_p_color',
 'ratio_std_pos',
 'ratio_std_stock',
 'diff_position',
 'ratio_std_shift1',
 'ratio_std_shift2']

#### Frequency encoding por grupos

Útil para series temporales

In [39]:
agrupacion = ['subfamily_id', 'size_id', 'color_id']
a = train.groupby(agrupacion)['product_id'].count()
a.name = 'N1'
train = train.join(a, on=agrupacion, how='left')

a = test.groupby(agrupacion)['product_id'].count()
a.name = 'N1'
test = test.join(a, on=agrupacion, how='left')

In [54]:
agrupacion = ['subfamily_id', 'size_id', 'color_id']
a = train.groupby(agrupacion)['product_id'].count()
a.name = 'N2'
train = train.join(a, on=agrupacion, how='left')

a = test.groupby(agrupacion)['product_id'].count()
a.name = 'N2'
test = test.join(a, on=agrupacion, how='left')

In [63]:
agrupacion = ['subfamily_id', 'family_id']
a = train.groupby(agrupacion)['product_id'].count()
a.name = 'N3'
train = train.join(a, on=agrupacion, how='left')

a = test.groupby(agrupacion)['product_id'].count()
a.name = 'N3'
test = test.join(a, on=agrupacion, how='left')

In [107]:
agrupacion = ['product_id']
a = train.groupby(agrupacion)['date_number'].count()
a.name = 'N5'
train = train.join(a, on=agrupacion, how='left')

a = test.groupby(agrupacion)['date_number'].count()
a.name = 'N5'
test = test.join(a, on=agrupacion, how='left')

In [115]:
agrupacion = ['product_id', 'size_id', 'color_id']
a = train.groupby(agrupacion)['product_id'].count()
a.name = 'N6'
train = train.join(a, on=agrupacion, how='left')

a = test.groupby(agrupacion)['product_id'].count()
a.name = 'N6'
test = test.join(a, on=agrupacion, how='left')

In [133]:
agrupacion = ['size_id', 'color_id']
a = train.groupby(agrupacion)['product_id'].count()
a.name = 'N7'
train = train.join(a, on=agrupacion, how='left')

a = test.groupby(agrupacion)['product_id'].count()
a.name = 'N7'
test = test.join(a, on=agrupacion, how='left')

In [162]:
agrupacion = ['product_id']
a = train.groupby(agrupacion)['family_id'].count()
a.name = 'N8'
train = train.join(a, on=agrupacion, how='left')

a = test.groupby(agrupacion)['family_id'].count()
a.name = 'N8'
test = test.join(a, on=agrupacion, how='left')

In [181]:
agrupacion = ['price', 'family_id', 'subfamily_id']
a = train.groupby(agrupacion)['product_id'].count()
a.name = 'N9'
train = train.join(a, on=agrupacion, how='left')

a = test.groupby(agrupacion)['product_id'].count()
a.name = 'N9'
test = test.join(a, on=agrupacion, how='left')

In [195]:
agrupacion = ['price', 'family_id', 'subfamily_id', 'size_id', 'color_id']
a = train.groupby(agrupacion)['product_id'].count()
a.name = 'N10'
train = train.join(a, on=agrupacion, how='left')

a = test.groupby(agrupacion)['product_id'].count()
a.name = 'N10'
test = test.join(a, on=agrupacion, how='left')

In [209]:
agrupacion = ['price', 'family_id', 'subfamily_id', 'size_id', 'color_id', 'day']
a = train.groupby(agrupacion)['product_id'].count()
a.name = 'N11'
train = train.join(a, on=agrupacion, how='left')

a = test.groupby(agrupacion)['product_id'].count()
a.name = 'N11'
test = test.join(a, on=agrupacion, how='left')

In [209]:
agrupacion = ['block_id', 'family_id', 'subfamily_id']
a = train.groupby(agrupacion)['product_id'].count()
a.name = 'N12'
train = train.join(a, on=agrupacion, how='left')

a = test.groupby(agrupacion)['product_id'].count()
a.name = 'N12'
test = test.join(a, on=agrupacion, how='left')

In [210]:
# train = train.drop('N10', axis=1)
# test = test.drop('N10', axis=1)

In [211]:
train.head()

,date_number,product_id,block_id,stock,std_stock,family_id,subfamily_id,size_id,color_id,position_max,...,N2,N3,N4,N5,N6,N7,N8,N9,N10,N11
0,0,310130,1726,1383,34.811328,0.007130,0.000598,7,1,80.0,...,80,250,15,64,61,36187,64,64,61,9
1,0,1178388,592,60,2.160247,0.049198,0.012761,4,1,19.0,...,2469,4787,171,64,64,66956,64,1323,752,115
2,0,1561460,1625,2373,55.438769,0.123964,0.023599,5,1,38.0,...,3165,9866,5,64,64,49495,64,7089,2551,401
3,0,1874414,1135,1686,20.463906,0.110784,0.016021,6,1,190.0,...,2686,5942,13,64,64,31410,64,1683,466,74
4,0,2436420,779,245,23.377339,0.025903,0.004497,5,1,NaN,...,922,1880,39,64,64,49495,64,175,98,15


# Prueba de modelo

In [256]:
drop_cols = ['sales', 'date_number', 'product_id', 'block_id',
             'position_max', 'position_min', 'std_position', 'ratio_std_pos', 'diff_position'
            ]

col_sel = [
 'std_stock_shift1',
 'std_stock_shift2',
 'ratio_std_stock',
 'ratio_std_shift1',
 'ratio_std_shift2',]

X = train.loc[:,[c for c in train.columns if c not in drop_cols]]
# X = train[col_sel]
y = train[['sales']]

In [257]:
from sklearn.model_selection import train_test_split

In [258]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [259]:
from sklearn.model_selection import StratifiedKFold
from sklearn.externals import joblib
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error

In [260]:
model_name = 'lgbm'

params = {'max_depth':7,
          'metric':'mae',
          'max_delta_step': 0.2,
          'n_estimators':50000,
          'learning_rate':0.1,
          'colsample_bytree':0.6,
          'objective':'regression',
          'n_jobs':8,
          'seed':42,
          'lambda_l1':0,
          'lambda_l2':0,
#           'max_bin': 14,
#           'bagging_fraction':0.8,
         }

lgb_model = lgb.LGBMRegressor(**params)

In [261]:
train_ids = X_train.index
k = 5
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

In [263]:
counter = 1
be = 0
ft_importances = np.zeros(X_train.shape[1])
for train_index, test_index in skf.split(train_ids, y_train):
    print('--- Fold k {}:'.format(counter))

    X_fit, X_val = X_train.iloc[train_index, :], X_train.iloc[test_index, :]
    y_fit, y_val = y_train.iloc[train_index], y_train.iloc[test_index]
    
    y_val = np.log1p(y_val)
    y_fit = np.log1p(y_fit)

    lgb_model.fit(X_fit,
                  y_fit,
                  eval_set=[(X_val, y_val)],
                  verbose=1000,
                  early_stopping_rounds=50)

    ft_importances += lgb_model.feature_importances_

    be += np.expm1(lgb_model.best_score_['valid_0']['l1'])
#     be += lgb_model.best_score_['valid_0']['l1']
#     print('Score en el test:',mean_absolute_error(lgb_model.predict(X_test), y_test),'ventas')
    y_preds = np.expm1(lgb_model.predict(X_test))
    y_preds = [int(round(x)) for x in y_preds]
    print('Score en el test:',round(mean_absolute_error(y_preds, y_test), 3),'"ventas" de error absoluto medio')
    print('Predicciones:', y_preds[:20])
    print('Ventas reales:', y_test[:20].sales.tolist(),'\n\n')
    
    counter += 1
    
# print('\n\nBEST SCORE MEAN:', be / k,'SALES :)')

print('IMPORTANCIA DE LAS VARIABLES:\n')
ft_importances += lgb_model.feature_importances_
imp = pd.DataFrame({'feature': X_train.columns, 'importance': ft_importances/k})
df_imp_sort = imp.sort_values('importance', ascending=False)
df_imp_sort

--- Fold k 1:
Training until validation scores don't improve for 50 rounds.
[1000]	valid_0's l1: 0.472994
[2000]	valid_0's l1: 0.460329
[3000]	valid_0's l1: 0.454882
[4000]	valid_0's l1: 0.451419
[5000]	valid_0's l1: 0.449104
[6000]	valid_0's l1: 0.447561
Early stopping, best iteration is:
[6015]	valid_0's l1: 0.447517
Score en el test: 5.964 "ventas" de error absoluto medio
Predicciones: [7, 0, 2, 8, 1, 32, 1, 16, 88, 203, 23, 11, 5, 1, 1, 0, 27, 0, 1, 3]
Ventas reales: [5, 0, 0, 7, 4, 27, 4, 22, 117, 176, 30, 4, 16, 7, 2, 0, 9, 1, 0, 4] 


--- Fold k 2:
Training until validation scores don't improve for 50 rounds.
[1000]	valid_0's l1: 0.475351
[2000]	valid_0's l1: 0.461653
[3000]	valid_0's l1: 0.455585
[4000]	valid_0's l1: 0.451973
[5000]	valid_0's l1: 0.449761
Early stopping, best iteration is:
[4982]	valid_0's l1: 0.449711
Score en el test: 5.958 "ventas" de error absoluto medio
Predicciones: [10, 0, 1, 8, 1, 26, 1, 17, 86, 190, 22, 9, 5, 1, 2, 0, 23, 1, 0, 5]
Ventas reales: [5, 0,

NameError: name 'columnas' is not defined

In [265]:
print('IMPORTANCIA DE LAS VARIABLES:\n')
ft_importances += lgb_model.feature_importances_
imp = pd.DataFrame({'feature': X_train.columns, 'importance': ft_importances/k})
df_imp_sort = imp.sort_values('importance', ascending=False)
df_imp_sort

IMPORTANCIA DE LAS VARIABLES:



,feature,importance
14,std_stock_shift2,14622.4
11,std_stock_shift1,14393.4
10,stock_lead1,13605.0
12,stock_lag2,12776.6
13,stock_lead2,12659.4
18,ratio_std_shift2,12536.0
17,ratio_std_shift1,12241.0
9,stock_lag1,12140.8
1,std_stock,11939.8
0,stock,11819.0


In [299]:
from sklearn.linear_model import Lasso

In [300]:
lasso = Lasso()

In [286]:
train_ids = X_train.index
k = 5
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

In [304]:
counter = 1
be = 0
for train_index, test_index in skf.split(train_ids, y_train):
    print('Fold k {}\n'.format(counter))

    X_fit, X_val = X_train.iloc[train_index, :], X_train.iloc[test_index, :]
    y_fit, y_val = y_train.iloc[train_index], y_train.iloc[test_index]
    
    X_fit = X_fit.fillna(-1)
    X_val = X_val.fillna(-1)
    
#     y_val = np.log1p(y_val)
#     y_fit = np.log1p(y_fit)
    
    lasso.fit(X_fit, y_fit)
    
    print('Score en el test:',mean_absolute_error(lasso.predict(X_test.fillna(-1)), y_test),'ventas')
#     print('Score en el test:',mean_absolute_error(np.expm1(lasso.predict(X_test.fillna(-1))), y_test),'ventas')
    
    counter += 1
    
    
# print('\n\nBEST SCORE MEAN:', be / k,'SALES :)')

Fold k 1

Score en el test: 11.051975266788306 ventas
Fold k 2

Score en el test: 11.129689549141098 ventas
Fold k 3

Score en el test: 11.109224964658654 ventas
Fold k 4

Score en el test: 11.096183497137874 ventas
Fold k 5

Score en el test: 11.151459385943244 ventas


In [64]:
from sklearn.model_selection import TimeSeriesSplit

In [67]:
tscv = TimeSeriesSplit(n_splits=5)

In [68]:
counter = 1
be = 0
for train_index, test_index in tscv.split(X):
    X_fit, X_val = X.iloc[train_index, :], X.iloc[test_index, :]
    y_fit, y_val = y.iloc[train_index], y.iloc[test_index]
    
#     y_val = np.log1p(y_val)
#     y_fit = np.log1p(y_fit)
    
    
    lgb_model.fit(X_fit,
                  y_fit,
                  eval_set=[(X_val, y_val)],
                  verbose=1000,
                  early_stopping_rounds=20)


#     be += np.expm1(lgb_model.best_score_['valid_0']['l1'])
    be += lgb_model.best_score_['valid_0']['l1']
    
#     print('Score en el test:',mean_absolute_error(lgb_model.predict(X_test), y_test),'ventas')
#     print('Score en el test:',mean_absolute_error(np.expm1(lgb_model.predict(X_test)), y_test),'ventas')
    
    counter += 1

Training until validation scores don't improve for 20 rounds.
[1000]	valid_0's l1: 14.9035
[2000]	valid_0's l1: 14.148
[3000]	valid_0's l1: 13.7446
Early stopping, best iteration is:
[3494]	valid_0's l1: 13.6114
Training until validation scores don't improve for 20 rounds.
[1000]	valid_0's l1: 12.3938
[2000]	valid_0's l1: 11.7195
[3000]	valid_0's l1: 11.3735
Early stopping, best iteration is:
[3420]	valid_0's l1: 11.2791
Training until validation scores don't improve for 20 rounds.
[1000]	valid_0's l1: 10.2273
Early stopping, best iteration is:
[1032]	valid_0's l1: 10.1833
Training until validation scores don't improve for 20 rounds.
Early stopping, best iteration is:
[823]	valid_0's l1: 9.79996
Training until validation scores don't improve for 20 rounds.
[1000]	valid_0's l1: 8.96911
Early stopping, best iteration is:
[1033]	valid_0's l1: 8.93757
